In [1]:
#Importing the libraries
import cv2
import requests
from ultralytics import YOLO
import matplotlib as plt

In [2]:
# Load YOLO11n model
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
# Telegram Bot Details
BOT_TOKEN = "..."  # Replace with your Telegram bot token
CHAT_ID = "..."  # Replace with your Telegram chat ID

In [4]:
# Define class names (Replace these with actual names and IDs)
CLASS_NAMES = {
    0: "Cabbage",
    1: "Capsicum",
    2: "orange",
    3: "tomato",
    4:"watermelon"
}

In [5]:
#Telegram related functions for message delivery
def send_telegram_alert(message):
    """Send a Telegram message"""
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    response = requests.post(url, json=payload)
    print("✅ Telegram alert sent!", response.json())

In [15]:
# Load and process the image
image_path = "capsicum.jpeg"  # Change this to your image
output_path = "Output images/capsicum.jpeg"
image = cv2.imread(image_path)
results = model(image)


0: 576x640 4 Capsicums, 81.8ms
Speed: 4.3ms preprocess, 81.8ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)


In [16]:
# Dictionary to count objects per category
category_counts = {name: 0 for name in CLASS_NAMES.values()}

In [17]:
# Process detections
for r in results:
    for box in r.boxes:
        class_id = int(box.cls[0])  # Get class index
        confidence = float(box.conf[0])  # Confidence score
        
        if class_id in CLASS_NAMES and confidence >= 0.3:
            category_name = CLASS_NAMES[class_id]
            category_counts[category_name] += 1  # Count objects
            
            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Assign a color (change as needed)
            color = (0, 255, 0) if category_name == "Tomato" else (255, 0, 0)
            
            # Draw bounding box and label
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, category_name, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display category counts on the image
y_offset = 40
for category, count in category_counts.items():
    cv2.putText(image, f"{category}: {count}", (20, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    y_offset += 30

# Save and display output image
cv2.imwrite(output_path, image)
print(f"✅ Output saved as {output_path}")

# Generate Telegram alert message
alert_message = "🔔 Detection Report:\n"
for category, count in category_counts.items():
    alert_message += f"{category}: {count}\n"

# Send alert if any category has less than 5 objects
if any(count < 5 for count in category_counts.values()):
    send_telegram_alert(alert_message)

✅ Output saved as Output images/capsicum.jpeg
✅ Telegram alert sent! {'ok': True, 'result': {'message_id': 7, 'from': {'id': 8188314211, 'is_bot': True, 'first_name': 'Task_detection', 'username': 'Task_detection_bot'}, 'chat': {'id': 1443202982, 'first_name': 'Utkarsh', 'last_name': 'Srivastava', 'type': 'private'}, 'date': 1741720969, 'text': '🔔 Detection Report:\nCabbage: 0\nCapsicum: 4\norange: 0\ntomato: 0\nwatermelon: 0'}}


In [18]:
cv2.imshow("Tomato Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()